```
bryan@ichabod mac_bucket % for file in *_production.json; do
  databricks fs cp "$file" dbfs:/Volumes/geodata/petra/production_raw/
done
```


In [ ]:
df = spark.read.format("json").load("/Volumes/geodata/petra/production_raw/")

display(df)

In [ ]:
from common.transforms import string_to_iso_date, generate_hash, replace_10e30_with_null
import pyspark.sql.functions as F

df_flat = df.select(
    F.col("repo_id"),
    F.col("uwi.uwi").alias("uwi"),
    F.col("uwi.wsn").alias("wsn"),
    F.col("mopddata.jan").alias("jan"),
    F.col("mopddata.feb").alias("feb"),
    F.col("mopddata.mar").alias("mar"),
    F.col("mopddata.apr").alias("apr"),
    F.col("mopddata.may").alias("may"),
    F.col("mopddata.jun").alias("jun"),
    F.col("mopddata.jul").alias("jul"),
    F.col("mopddata.aug").alias("aug"),
    F.col("mopddata.sep").alias("sep"),
    F.col("mopddata.oct").alias("oct"),
    F.col("mopddata.nov").alias("nov"),
    F.col("mopddata.dec").alias("dec"),
    F.col("mopddata.chgdate").alias("app_row_changed"),
    F.col("mopddata.cum").alias("cum"),
    F.col("mopddata.mid").alias("mid"),
    F.col("mopddata.year").alias("year"),
    F.col("mopddef.name").alias("name"),
    F.col("mopddef.desc").alias("desc"),
    F.col("mopddef.units").alias("units"),
)

df_production = df_flat


# enforce timestamp for dates
for col_name in ["app_row_changed"]:
    df_production = string_to_iso_date(df_production, col_name, col_name)


# ensure real nulls
for col_name in [
    "jan",
    "feb",
    "mar",
    "apr",
    "may",
    "jun",
    "jul",
    "aug",
    "sep",
    "oct",
    "nov",
    "dec",
    "cum",
]:
    df_production = replace_10e30_with_null(df_production, col_name, col_name)


# add id hash
id_columns = ["repo_id", "uwi"]
df_production = generate_hash(df_production, "id", "production", *id_columns)


display(df_production)

In [ ]:
from common.transforms import upsert_dataframe_to_table

result = upsert_dataframe_to_table(df_production, "geodata.petra.production_bronze")
display(result)